# Homework 4: Sequence to Sequence Modeling

The aim of this homework is to familiarize you with sequence-to-sequence language modeling, specifically using an encoder-decoder model. This is the coding portion; you also have a written portion. The written portion can be found the homework instructions, i.e. the pdf you download from the syllabus website. In this notebook, you are provided with pre-written code for a simple sequence-to-sequence model that already works and learns how to reverse short sequences of numbers.

If you run this whole jupyter notebook, it will learn to reverse short sequences of numbers. Although much of this code you will not be modifying, we recommend reading through it to get a sense of how the model and training works.

This starter code is based on [this tutorial](https://pytorch.org/tutorials/intermediate/seq2seq_translation_tutorial.html) by Sean Robertson from the PyTorch website. It has been modified by Katy Ilonka Gero for COMS W4705 taught at Columbia University by Professor Kathy McKeown. 

### Overview

Your assignment is to:

1. modify this code to run with the E2E restaurant data set (provided)
2. train a model on this dataset on a GPU
3. implement beam search for evaluation
4. implement a BLEU evaluator and report BLEU scores

These do not need to be done in this order.

You must submit:

1. This jupyter notebook, with your solutions to the above assignments. (Note cells that require specific outputs. Do not clear outputs before submitting.)
2. A saved model (encoder and decoder) that takes in a meaning representation and generates a restaurant description.

Write all your code **in this jupyter notebook**. Cells are provided where you should be implementing your code. See homework instructions for further details on how to submit this homework.

### 1. Modify to work with E2E Dataset

You will be working with the end-to-end (E2E) challenge dataset. More information can be found on [their website](http://www.macs.hw.ac.uk/InteractionLab/E2E/). In this dataset, the inputs are restaurant meaning representations, which are a series of key-value pairs that encode information about a restaurant. The outputs are fluent sentences that describe the restaurant. Here is an example:

*Input: Meaning Representation*

```
name[The Eagle],
eatType[coffee shop],
food[French],
priceRange[moderate],
customerRating[3/5],
area[riverside],
kidsFriendly[yes],
near[Burger King]
```

*Output: Fluent Sentence*

```
The three star coffee shop, The Eagle, gives families a mid-priced dining experience featuring a variety of wines and cheeses. Find The Eagle near Burger King.
```

You will need to read in and process the training and development data. This data is provided in csv format. Here is an example line from the `trainset.csv` file:

```
"name[Browns Cambridge], eatType[coffee shop], food[English], customer rating[5 out of 5], area[riverside], familyFriendly[no], near[Crowne Plaza Hotel]","Browns Cambridge, a 5 out of 5 English coffee shop, is not kid friendly. It is located near Crowne Plaza Hotel and riverside."
```

You will need to tokenize the input and output. The input should be tokenized such that each token is a single entry from the meaning representation. You can decide how to tokenize the output. Here is how the input should be tokenize, and a simple tokenization for the output:

*Input:*

```
['name[Browns Cambridge]', 'eatType[coffee shop]', 'food[English]', 'customer rating[5 out of 5]', 'area[riverside]', 'familyFriendly[no]', 'near[Crowne Plaza Hotel]']
```

*Output:*

```
['<SOS>', 'Browns', 'Cambridge,', 'a', '5', 'out', 'of', '5', 'English', 'coffee', 'shop,', 'is', 'not', 'kid', 'friendly.', 'It', 'is', 'located', 'near', 'Crowne', 'Plaza', 'Hotel', 'and', 'riverside.', '<EOS>']
```

Be sure to note the `<SOS>` (start-of-sequence) and `<EOS>` (end-of-sequence) tokens in the output. This is important and necessary! The decoder requires start and end tokens; the start token gives it an initial input to start generating, and the end token lets you know when to stop.

Your first goal is to load in this data with [torchtext](https://torchtext.readthedocs.io/en/latest/index.html), a library used to manage text datasets in pytorch. *You do not need to change anything in the model or training or evaluation.* All you need to do is load in the data similar to how the number-reversal data is loaded in.

### 2. Train a model on this data

To train a model on this data in a reasonable period of time, you will need to run this notebook on the Google Cloud VM with a GPU. [This tutorial](https://towardsdatascience.com/running-jupyter-notebook-in-google-cloud-platform-in-15-min-61e16da34d52) gives a good explanation of how to use jupyter notebooks from a Google Cloud VM. It can take time to correctly get set up on a GPU, so don't leave this to the last minute. 

However, we *do* recommend testing your code by loading in a small amount of data (say, 5 examples,) and training on these. This should train quickly even without a GPU and the model should be able to almost memorize the data. This is generally good practice with generative networks -- ensure your model will memorize a small amount of data.

With the full e2e dataset on a GPU, it should take around 20 minutes to train a single epoch. You should see decent results after a single epoch. Decent results are sentences with a few mistakes, but are mostly readable. You are encouraged to see what kind of improvements can be found with more training or different parameters.

You do not need to modify any code to train the model, nor are you allowed to modify the model. You may modify the `trainIters` function, if you would like to improve how you track progress, or change parameters while training. For example, it can be useful to decrease the teacher-forcing ratio as training progresses.

*You must submit a trained model. This model must be a GPU model. It is not reasonable to train this model with a CPU; part of the assignment is training it on a GPU.*

*Note that the model is trained using single examples -- that is, it doesn't use batching. Batching is possible with seq2seq models, but for simplicity of reading the code we have not implemented it here.*

### 3. Implement a beam search evaluator

We provide you with an evaluation function that takes in an input sequence and generates an output sentence given a trained model. This evaluation function performs *greedy decoding* by taking the most likely token at each generation step. You are required to implement a beam search version of this evaluation function, that keeps track of the top *k* most likely sequences at each generation step, and then returns the top *k* best sequences with their associated probabilities.

### 4. Implement a BLEU evaluator and report scores

While loss and accuracy are good for tracking training progress, they don't tell us much about how well the model generates meaningful sentences. You need to implement a BLEU evaluation function that takes in an input/output pair and returns the BLEU score for how well the model predicts the output.

You can find a formal description of how to calculate BLEU in the original paper, [BLEU: A Method for Automatic Evaluation of Machine Translation](https://www.aclweb.org/anthology/P02-1040.pdf). We reprodue this formal description for you in the homework instructions.

When reporting these scores, use the *development dataset* provided. Report scores for greedy decoding and beam search (beam size=3). For beam search, use the top-scoring output sentence as the score for that datapoint.

*You must implement your BLEU evaluator from scratch.* There exist python libraries that implement BLEU for you. Do not use these.

### Don't forget the written portion!

A series of open-ended questions about these tasks are required for the written portion of this homework. Please see the homework instructions for this, as well as instructions about how to submit.

In [1]:
# You may modify this cell

import random
import time

import torch
import torch.nn as nn
from torch import optim
import torch.nn.functional as F

import torchtext

In [2]:
# DO NO MODIFY

# this is useful for checking if your code is successfully using the GPU

mydevice = torch.device("cuda" if torch.cuda.is_available() else "cpu")
mydevice

device(type='cuda')

In [3]:
# DO NOT MODIFY

SOS_TOKEN = '<sos>'
EOS_TOKEN = '<eos>'

MAX_LEN = 50

def len_filter(example):
    return len(example.src) <= MAX_LEN and len(example.tgt) <= MAX_LEN

### Load dummy number reversal dataset

In [4]:
# DO NOT MODIFY

train_path = 'data/toy_reverse/train/data.txt'
dev_path = 'data/toy_reverse/dev/data.txt'

src = torchtext.data.Field(
    batch_first=True, 
    include_lengths=True
    )
tgt = torchtext.data.Field(
    batch_first=True, 
    preprocessing = lambda seq: [SOS_TOKEN] + seq + [EOS_TOKEN]
    )

data_train = torchtext.data.TabularDataset(
        path=train_path, format='tsv',
        fields=[('src', src), ('tgt', tgt)],
        filter_pred=len_filter
    )
data_dev = torchtext.data.TabularDataset(
        path=dev_path, format='tsv',
        fields=[('src', src), ('tgt', tgt)],
        filter_pred=len_filter
    )

### 1. Load the e2e data

Load in the E2E data similar to how the dummy number reversal dataset is loaded. That is, use the same `torchtext.data.Field` and `torchtext.data.TabularDataset` classes.

In [6]:
# WRITE YOUR CODE FOR LOADING THE E2E DATA HERE

USE_BABY = False

def comma_split(item):
    return [x.strip() for x in item.split(',')]

if USE_BABY:
    train_path = 'data/e2e-dataset/baby-train.csv'
    dev_path = 'data/e2e-dataset/baby-dev.csv'
else:
    train_path = 'data/e2e-dataset/trainset.csv'
    dev_path = 'data/e2e-dataset/devset.csv'

src = torchtext.data.Field(
    batch_first=True, 
    include_lengths=True,
    tokenize=comma_split
    )
tgt = torchtext.data.Field(
    batch_first=True, 
    preprocessing = lambda seq: [SOS_TOKEN] + seq + [EOS_TOKEN]
    )

data_train = torchtext.data.TabularDataset(
        path=train_path, format='csv',
        fields=[('src', src), ('tgt', tgt)],
        filter_pred=len_filter,
        skip_header=True
    )
data_dev = torchtext.data.TabularDataset(
        path=dev_path, format='csv',
        fields=[('src', src), ('tgt', tgt)],
        filter_pred=len_filter,
        skip_header=True
    )

Have a look at the vocab and some example data points.

*If you have loaded in the E2E dataset correctly, the code in the cell below should work without any modification.*

In [7]:
# You may modify this cell

src.build_vocab(data_train, max_size=50000)
tgt.build_vocab(data_train, max_size=50000)
input_vocab = src.vocab
output_vocab = tgt.vocab

print('20 tokens from input vocab:\n', list(input_vocab.stoi.keys())[:20])
print('\n20 tokens from output vocab:\n', list(output_vocab.stoi.keys())[:20])

print('\nnum training examples:', len(data_train.examples))

item = random.choice(data_train.examples)
print('\nexample train data:')
print('src:\n', item.src)
print('tgt:\n', item.tgt)

20 tokens from input vocab:
 ['<unk>', '<pad>', 'familyFriendly[yes]', 'area[riverside]', 'eatType[coffee shop]', 'familyFriendly[no]', 'area[city centre]', 'eatType[pub]', 'food[Japanese]', 'food[Italian]', 'food[Fast food]', 'food[French]', 'priceRange[moderate]', 'priceRange[less than £20]', 'customer rating[average]', 'customer rating[low]', 'priceRange[high]', 'customer rating[5 out of 5]', 'priceRange[more than £30]', 'food[Indian]']

20 tokens from output vocab:
 ['<unk>', '<pad>', 'is', '<eos>', '<sos>', 'a', 'The', 'the', 'in', 'near', 'of', 'and', 'food', 'customer', 'located', 'It', 'restaurant', 'has', 'coffee', 'price']

num training examples: 42037

example train data:
src:
 ['name[The Waterman]', 'food[Indian]', 'priceRange[high]', 'customer rating[1 out of 5]', 'area[city centre]', 'familyFriendly[yes]']
tgt:
 ['<sos>', 'The', 'Waterman', 'Indian', 'food', 'has', 'a', 'low', 'customer', 'rating', 'and', 'a', 'family', 'friendly', 'environment', 'with', 'a', 'central', '

### Model definition and training functions

In [8]:
# DO NOT MODIFY

class EncoderRNN(nn.Module):
    def __init__(self, input_size, hidden_size):
        super(EncoderRNN, self).__init__()
        self.hidden_size = hidden_size

        self.embedding = nn.Embedding(input_size, hidden_size)
        self.gru = nn.GRU(hidden_size, hidden_size)

    def forward(self, myinput, hidden):
        embedded = self.embedding(myinput).view(1, 1, -1)
        output = embedded
        output, hidden = self.gru(output, hidden)
        return output, hidden

    def initHidden(self):
        return torch.zeros(1, 1, self.hidden_size, device=mydevice)

    
class DecoderRNN(nn.Module):
    def __init__(self, hidden_size, output_size):
        super(DecoderRNN, self).__init__()
        self.hidden_size = hidden_size

        self.embedding = nn.Embedding(output_size, hidden_size)
        self.gru = nn.GRU(hidden_size, hidden_size)
        self.out = nn.Linear(hidden_size, output_size)
        self.softmax = nn.LogSoftmax(dim=1)

    def forward(self, input, hidden):
        output = self.embedding(input).view(1, 1, -1)
        output = F.relu(output)
        output, hidden = self.gru(output, hidden)
        output = self.softmax(self.out(output[0]))
        return output, hidden

    def initHidden(self):
        return torch.zeros(1, 1, self.hidden_size, device=mydevice)

In [9]:
# DO NOT MODIFY

def train(input_tensor, target_tensor, encoder, decoder, encoder_optimizer, decoder_optimizer, criterion,
          max_length=MAX_LEN, teacher_forcing_ratio=0.5):
    
    # get an initial hidden state for the encoder
    encoder_hidden = encoder.initHidden()

    # zero the gradients of the optimizers
    encoder_optimizer.zero_grad()
    decoder_optimizer.zero_grad()

    # get the seq lengths, used for iterating through encoder/decoder
    input_length = input_tensor.size(0)
    target_length = target_tensor.size(0)

    # create empty tensor to fill with encoder outputs
    encoder_outputs = torch.zeros(max_length, encoder.hidden_size, device=mydevice)

    # create a variable for loss
    loss = 0
    
    # pass the inputs through the encoder
    for ei in range(input_length):
        encoder_output, encoder_hidden = encoder(input_tensor[ei], encoder_hidden)
        encoder_outputs[ei] = encoder_output[0, 0]

    # create a start-of-sequence tensor for the decoder
    decoder_input = torch.tensor([[output_vocab.stoi[SOS_TOKEN]]], device=mydevice)

    # set the decoder hidden state to the final encoder hidden state
    decoder_hidden = encoder_hidden

    # decide if we will use teacher forcing
    use_teacher_forcing = True if random.random() < teacher_forcing_ratio else False

    for di in range(target_length):
        decoder_output, decoder_hidden = decoder(decoder_input, decoder_hidden)
        
        topv, topi = decoder_output.topk(1)
        decoder_input = topi.squeeze().detach()  # detach from history as input
                
        loss += criterion(decoder_output, target_tensor[di].unsqueeze(0))
        
        if use_teacher_forcing:
            decoder_input = target_tensor[di]
        
        if decoder_input.item() == output_vocab.stoi[EOS_TOKEN]:
                break

    loss.backward()

    encoder_optimizer.step()
    decoder_optimizer.step()

    return loss.item() / target_length

In [10]:
# You may modify this cell

def trainIters(encoder, decoder, n_iters, print_every=1000, learning_rate=0.01, teacher_forcing_ratio=0.5):
    print(f'Running {n_iters} epochs...')
    print_loss_total = 0
    print_loss_epoch = 0

    encoder_optim = optim.SGD(encoder.parameters(), lr=learning_rate)
    decoder_optim = optim.SGD(decoder.parameters(), lr=learning_rate)

    # note batch size of 1, just for simplicity
    # DO NOT INCREASE THE BATCH SIZE
    batch_iterator = torchtext.data.Iterator(
        dataset=data_train, batch_size=1,
        sort=False, sort_within_batch=True,
        sort_key=lambda x: len(x.src),
        device=mydevice, repeat=False)
    

    criterion = nn.NLLLoss()

    for e in range(n_iters):
        batch_generator = batch_iterator.__iter__()
        step = 0
        start = time.time()
        for batch in batch_generator:
            step += 1
            
            # get the input and target from the batch iterator
            input_tensor, input_lengths = getattr(batch, 'src')
            target_tensor = getattr(batch, 'tgt')
            
            # this is because we're not actually using the batches.
            # batch size is 1 and this just selects that first one
            input_tensor = input_tensor[0]
            target_tensor = target_tensor[0]

            loss = train(input_tensor, target_tensor, encoder, decoder, encoder_optim, decoder_optim, criterion, teacher_forcing_ratio=teacher_forcing_ratio)
            print_loss_total += loss
            print_loss_epoch += loss
            

            if step % print_every == 0:
                print_loss_avg = print_loss_total / print_every
                print_loss_total = 0
                t = (time.time() - start) / 60
                print(f'step: {step}\t avg loss: {print_loss_avg:.2f}\t time for {print_every} steps: {t:.2f} min')
                start = time.time()
        
        print_loss_avg = print_loss_epoch / step
        print_loss_epoch = 0
        print(f'End of epoch {e}, avg loss {print_loss_avg:.2f}')  

### 2. Create and train a model

In [10]:
# You may modify this cell

hidden_size = 128
encoder1 = EncoderRNN(len(input_vocab), hidden_size).to(mydevice)
decoder1 = DecoderRNN(hidden_size, len(output_vocab)).to(mydevice)

Here are some guidelines for how much training to expect. Note that these *guidelines*; they are not exact.

Only 1 epoch is needed for the number reversal dataset. This produces near-perfect results, and should take less than 5 minutes to run on a CPU.

To memorize ~5 examples of the e2e dataset, ~100 epochs are needed (with a high teacher forcing ratio). This produces near-perfect results.

To train on the full e2e dataset, only 1 epoch is needed to see decent outputs on the training data. More are required to increase fluency and see improvements on the development data.

In [11]:
# You may modify this cell
# but be sure that it prints some indication of how training is progressing

trainIters(encoder1, decoder1, 100, print_every=1000, teacher_forcing_ratio=0.75)

Running 100 epochs...
End of epoch 0, avg loss 4.06
End of epoch 1, avg loss 4.10
End of epoch 2, avg loss 3.73
End of epoch 3, avg loss 3.84
End of epoch 4, avg loss 3.78
End of epoch 5, avg loss 3.47
End of epoch 6, avg loss 3.40
End of epoch 7, avg loss 3.27
End of epoch 8, avg loss 2.38
End of epoch 9, avg loss 2.49
End of epoch 10, avg loss 2.65
End of epoch 11, avg loss 2.93
End of epoch 12, avg loss 2.60
End of epoch 13, avg loss 2.54
End of epoch 14, avg loss 2.20
End of epoch 15, avg loss 2.15
End of epoch 16, avg loss 2.00
End of epoch 17, avg loss 2.34
End of epoch 18, avg loss 2.01
End of epoch 19, avg loss 2.02
End of epoch 20, avg loss 1.86
End of epoch 21, avg loss 1.71
End of epoch 22, avg loss 2.49
End of epoch 23, avg loss 2.04
End of epoch 24, avg loss 2.19
End of epoch 25, avg loss 2.04
End of epoch 26, avg loss 1.35
End of epoch 27, avg loss 1.96
End of epoch 28, avg loss 1.59
End of epoch 29, avg loss 1.41
End of epoch 30, avg loss 1.25
End of epoch 31, avg loss 1

In [11]:
# WRITE YOUR CODE FOR SAVING YOUR MODEL HERE

enc_path = 'saved-models/gpu-encoder1-e2e-1epoch.mdl'
dec_path = 'saved-models/gpu-decoder1-e2e-1epoch.mdl'

In [13]:
torch.save(encoder1, enc_path)
torch.save(decoder1, dec_path)

/anaconda3/envs/py37/lib/python3.7/site-packages/torch-1.2.0-py3.7-macosx-10.9-x86_64.egg/torch/serialization.py:256: UserWarning: Couldn't retrieve source code for container of type EncoderRNN. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
/anaconda3/envs/py37/lib/python3.7/site-packages/torch-1.2.0-py3.7-macosx-10.9-x86_64.egg/torch/serialization.py:256: UserWarning: Couldn't retrieve source code for container of type DecoderRNN. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "


In [12]:
encoder1 = torch.load(enc_path)
decoder1 = torch.load(dec_path)

In [13]:
# DO NOT MODIFY

def evaluate(encoder, decoder, sentence, max_length=MAX_LEN):
    with torch.no_grad():
        input_tensor = torch.tensor([input_vocab.stoi[word] for word in sentence], device=mydevice)
        input_length = input_tensor.size()[0]
        encoder_hidden = encoder.initHidden()

        encoder_outputs = torch.zeros(max_length, encoder.hidden_size, device=mydevice)

        for ei in range(input_length):
            encoder_output, encoder_hidden = encoder(input_tensor[ei], encoder_hidden)
            encoder_outputs[ei] += encoder_output[0, 0]

        decoder_input = torch.tensor([[output_vocab.stoi[SOS_TOKEN]]], device=mydevice)

        decoder_hidden = encoder_hidden

        decoded_words = []

        for di in range(max_length):
            decoder_output, decoder_hidden = decoder(decoder_input, decoder_hidden)
            topv, topi = decoder_output.data.topk(1)
            next_word = output_vocab.itos[topi.item()]
            decoded_words.append(next_word)
            if next_word == EOS_TOKEN:
                break

            decoder_input = topi.squeeze().detach()

        return decoded_words

### 3. Implement beam search evaluator

Be sure to return all the output sequences (i.e. if the beam size is k, you should return k sequences) and their associated probabilities. You will need the associated probabilities to select the best performing sequence when calculating BLEU.

In [22]:
# WRITE YOUR CODE FOR BEAM SEARCH HERE

# The output of this cell should be an example input from the dev set, 
# and three outputs from a beam search evaluator.

def evaluate_with_beam(encoder, decoder, sentence, k=3, max_length=MAX_LEN):
    with torch.no_grad():
        input_tensor = torch.tensor([input_vocab.stoi[word] for word in sentence], device=mydevice)
        input_length = input_tensor.size()[0]
        encoder_hidden = encoder.initHidden()

        encoder_outputs = torch.zeros(max_length, encoder.hidden_size, device=mydevice)

        for ei in range(input_length):
            encoder_output, encoder_hidden = encoder(input_tensor[ei], encoder_hidden)
            encoder_outputs[ei] += encoder_output[0, 0]
        
        sos = output_vocab.stoi[SOS_TOKEN]
        # take first step in decoding
        decoder_input = torch.tensor([[sos]], device=mydevice)
        decoder_hidden = encoder_hidden
        decoder_output, decoder_hidden = decoder(decoder_input, decoder_hidden)
        topv, topi = decoder_output.data.topk(k)

        k_sequences = [[i] for i in topi[0].tolist()]  # will be lists of sequences of word indeces
        k_scores = [[i] for i in topv[0].tolist()]  # will be lists of scores for each word in each seq
        k_hidden = [decoder_hidden]*k  # will be the hidden states of each sequence
        
        for di in range(max_length):
            
            if all(output_vocab.itos[k_seq[-1]] == EOS_TOKEN for k_seq in k_sequences):
                break

            k_sequences_new = []
            k_scores_new = []
            k_hidden_new = []
            
            for ki, k_seq in enumerate(k_sequences):
                if output_vocab.itos[k_seq[-1]] == EOS_TOKEN:
                    continue
                
                decoder_input = torch.tensor([[k_seq[-1]]], device=mydevice)
                decoder_hidden = k_hidden[ki]
                
                decoder_output, decoder_hidden = decoder(decoder_input, decoder_hidden)
                
                topv, topi = decoder_output.data.topk(k)
                
                k_sequences_new += [k_seq + [i] for i in topi[0].tolist()]
                k_scores_new += [k_scores[ki] + [i] for i in topv[0].tolist()] 
                k_hidden_new += [decoder_hidden]*k
            
            k_scores_overall = [sum(k_score)/len(k_score) for k_score in k_scores_new]
            topk = sorted(range(len(k_scores_overall)), key=lambda i: k_scores_overall[i])[-k:]

            k_sequences = [k_sequences_new[i] for i in topk]
            k_scores = [k_scores_new[i] for i in topk]
            k_hidden = [k_hidden_new[i] for i in topk]

    final_scores = [sum(k_score)/len(k_score) for k_score in k_scores]
    
    decoded_words = []
    for k_seq in k_sequences:
        decoded_words.append([output_vocab.itos[i] for i in k_seq])
    
    return decoded_words, final_scores

item = random.choice(data_train.examples)
seq = item.src
print(seq)

output = ' '.join(evaluate(encoder1, decoder1, seq))
print(f'\ngreedy:\n{output}')

k_words, k_scores = evaluate_with_beam(encoder1, decoder1, seq, k=3)
print('\nbeam:')
for words, score in zip(k_words, k_scores):
    output = ' '.join(words)
    print(f'{score:.4f} {output}')
print()

['name[The Eagle]', 'eatType[coffee shop]', 'food[Fast food]', 'priceRange[moderate]', 'customer rating[1 out of 5]', 'area[riverside]', 'familyFriendly[no]', 'near[Burger King]']

greedy:
<sos> The Eagle coffee shop is a fast food coffee shop near Burger King and has a moderate price range and a 1 out of of 5 customer rating. <eos>

beam:
-0.8448 <sos> The Eagle is a fast food coffee shop located near Burger King in riverside. It has a moderate price range and a customer rating of 1 out of 5. It is not kid friendly and has a moderate price range and a moderate price range and a moderate price range
-0.8430 <sos> The Eagle is a fast food coffee shop located near Burger King in riverside. It has a moderate price range and a customer rating of 1 out of 5. It is not kid friendly and has a moderate price range and a moderate price range and a moderate price range.
-0.8338 <sos> The Eagle is a fast food coffee shop located near Burger King in riverside. It has a moderate price range and a c

### 4. Implement BLEU evaluator

Remember that when calculating BLEU using beam search, select the top-scoring sequence output using the model probability.

In [15]:
# WRITE YOUR CODE FOR THE BLEU EVALUATION HERE

# The output of this cell should be the average BLEU score on the dev set
# for greedy decoding AND for beam search decoding (beam size = 3)

import numpy as np
from collections import Counter

def get_ngrams(sent, n):
    sent = sent.lower()
    tokens = [t for t in sent.split(' ') if t != '']
    ngrams = zip(*[tokens[i:] for i in range(n)])
    return [" ".join(ngram) for ngram in ngrams]
    
    
def modified_precision(refs, candidate):
    """
    assumes refs and candidate are lists of strings of desired ngram.
    refs is list of lists of strings.
    candidate is list of strings.
    """
    cand_counts = {}
    ref_counters = [Counter(ref) for ref in refs]
    
    for ngram in set(candidate):
        count = 0
        for ref_c in ref_counters:
            if ref_c[ngram] > count:
                count = ref_c[ngram]
        cand_counts[ngram] = count
    
    final_count = 0
    for ngram in cand_counts:
        final_count += min(candidate.count(ngram), cand_counts[ngram])

    return final_count / len(candidate)
        
                
def geo_mean(refs, candidate, n=4):
    """assumes refs and candidate are strings (refs is list of strings)"""
    geomean = 0
    for i in range(1, n+1):
        refs_n = [get_ngrams(ref, i) for ref in refs]
        cand_n = get_ngrams(candidate, i)
        p = modified_precision(refs_n, cand_n)
        if p == 0:
            break
        geomean += np.log(p)
    
    return np.exp((1/i) * geomean)

def brev_pen(refs, candidate):
    """assumes refs and candidate are strings (refs is list of strings)"""
    diffs = []
    hyp_len = len(candidate.lower().split(' '))
    for ref in refs:
        r = ref.lower().split(" ")
        diffs.append(abs(len(r) - hyp_len))
    closest_ref_len = len(refs[diffs.index(min(diffs))].split(' '))
    if hyp_len > closest_ref_len:
        return 1
    elif hyp_len == 0:
        return 0
    else:
        return np.exp(1 - (closest_ref_len / hyp_len))

def bleu(refs, candidate):
    """assumes refs and candidate are strings (refs is list of strings)"""
    geomean = geo_mean(refs, candidate)
    bp = brev_pen(refs, candidate)
    return bp * geomean

In [25]:
seqs = [data_dev.examples[0].src]
refs = [[' '.join(data_dev.examples[0].tgt)]]
for ex in data_dev.examples[1:]:
    if ex.src == seqs[-1]:
        refs[-1].append(' '.join(ex.tgt))
    else:
        seqs.append(ex.src)
        refs.append([' '.join(ex.tgt)])

        
bleu_scores_gr = []
bleu_scores_bm = []
for seq, rf in zip(seqs, refs):
    greedy = ' '.join(evaluate(encoder1, decoder1, seq))
    output, scores = evaluate_with_beam(encoder1, decoder1, seq, k=3)
    beam = ' '.join(output[scores.index(max(scores))])
    bleu_scores_gr.append(bleu(rf, greedy))
    bleu_scores_bm.append(bleu(rf, beam))

print(f'BLEU for greedy decoding: {sum(bleu_scores_gr)/len(seqs):.4f}')
print(f'BLEU for beam search: {sum(bleu_scores_bm)/len(seqs):.4f}')

BLEU for greedy decoding: 0.4537
BLEU for beam search: 0.3389


In [20]:
n=5

# for seq, rf in zip(seqs[1:n], refs[1:n]):
for _ in range(n):
    i = random.choice(range(len(seqs)))
    seq = seqs[i]
    rf = refs[i]
    greedy = ' '.join(evaluate(encoder1, decoder1, seq))
    output, scores = evaluate_with_beam(encoder1, decoder1, seq, k=3)
    beam = ' '.join(output[scores.index(max(scores))])
    b_gr = bleu(rf, greedy)
    b_bm = bleu(rf, beam)
    print('REFS:')
    for r in rf:
        print(r)
    print(f'GR: {b_gr:.4f} {greedy}')
    print(f'BM: {b_bm:.4f} {beam}')
    print()

REFS:
<sos> An average customer rated restaurant that is also child-friendly is The Wrestlers. <eos>
<sos> Looking for a family friendly venue with an average customer rating, then you need to check out The Wrestlers. <eos>
<sos> The Wrestlers has average Rating and is family-Friendly. <eos>
<sos> The Wrestlers has an average customer rating and is also children friendly. <eos>
<sos> The Wrestlers is children friendly and has an average rating <eos>
<sos> The Wrestlers is an average rated child friendly place. <eos>
<sos> The Wrestlers is an average family friendly place. <eos>
<sos> One family friendly venue is The Wrestlers. It is averagely rated. Near Café Rouge is a Japanese, family friendly place called The Golden Curry. It is riverside and has high customer ratings. <eos>
<sos> A nice place to take the family is The Wrestlers. <eos>
<sos> An average restaurant named The Wrestlers is children friendly. <eos>
<sos> The Wrestlers is an average, family friendly venue. Near Café Rouge